<a href="https://colab.research.google.com/github/davisdw/Lending_Tree_Loan_Prediction_Analysis/blob/main/pyspark_data_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing & Exporting CSV Data

**Reads the CSV files, accepted & rejected loans from Amazon AWS s3 Bucket**

**Reduce Un-needed Columns from both tables**

**Review the dataset and perform data wrangling and cleaning**

**Merge two datasets together**

**Export the cleaned_df dataset back to s3 bucket to prep for running modeling, prediction and visualization**

In [1]:
import boto3
import pandas as pd
from io import StringIO # uses this library for data conversion
import awsKeyConfig
import io
import numpy as np

# Use boto3 to get the object from S3
# Retrieve the aws credential keys

s3 = boto3.client('s3',
aws_access_key_id=awsKeyConfig.keyID,
aws_secret_access_key=awsKeyConfig.secretKey,
region_name='us-east-1'
)

**Data Wrangling and Cleaning for Loan Accepted Dataset**

In [2]:
# Retrieves the Bucket Name and Key <file_name> and 
obj = s3.get_object(Bucket='davis-data-cloud-of-wonders', Key='accepted_2007_to_2018Q4.csv')
data = obj['Body'].read().decode('utf-8')

# Validates whether the connection to s3 is successfull or fail 
status = obj.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 put_object response. Status - {status}")
else:
    print(f"Unsuccessful S3 put_object response. Status - {status}")


# Use StringIO to convert the string data to a file-like object
data_file = StringIO(data)

# Create a DataFrame from the CSV data
accepted_df = pd.read_csv(data_file)

accepted_df.head()


Successful S3 put_object response. Status - 200


/var/folders/xx/krls_17j0hjbfnsbh06vynnm0000gn/T/ipykernel_81852/987164762.py:18: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df = pd.read_csv(data_file)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# view number of columns and rows
accepted_df.shape

(2260701, 151)

In [4]:
# selected the required columns needed for analysis
clean_accept_df = accepted_df[["loan_amnt",
                    "term",
                    "int_rate",
                    "installment",
                    "grade",
                    "sub_grade",
                    "emp_title",
                    "emp_length",
                    "home_ownership",
                    "annual_inc",
                    "verification_status",
                    "issue_d",
                    "loan_status",
                    "purpose",
                    "addr_state",
                    "dti",
                    "fico_range_low",
                    "fico_range_high" ]]

clean_accept_df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,fico_range_low,fico_range_high
0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,PA,5.91,675.0,679.0
1,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,SD,16.06,715.0,719.0
2,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,home_improvement,IL,10.78,695.0,699.0
3,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,debt_consolidation,NJ,17.06,785.0,789.0
4,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,major_purchase,PA,25.37,695.0,699.0


In [5]:
# view the dataset shape after removal of un-needed columns:
clean_accept_df.shape

(2260701, 18)

In [6]:
# Take two fico scores columns and find an average for the credit score and place it in new columns
clean_accept_df["fico_score"] = clean_accept_df[["fico_range_low", "fico_range_high"]].mean(axis=1)
clean_accept_df.head()

/var/folders/xx/krls_17j0hjbfnsbh06vynnm0000gn/T/ipykernel_81852/1671030014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accept_df["fico_score"] = clean_accept_df[["fico_range_low", "fico_range_high"]].mean(axis=1)


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,fico_range_low,fico_range_high,fico_score
0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,PA,5.91,675.0,679.0,677.0
1,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,SD,16.06,715.0,719.0,717.0
2,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,home_improvement,IL,10.78,695.0,699.0,697.0
3,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,debt_consolidation,NJ,17.06,785.0,789.0,787.0
4,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,major_purchase,PA,25.37,695.0,699.0,697.0


In [7]:
clean_accept_df = clean_accept_df.drop(columns=["fico_range_low", "fico_range_high"])


In [8]:
# view full display of the data types for the accepted table
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(clean_accept_df.dtypes)



loan_amnt              float64
term                    object
int_rate               float64
installment            float64
grade                   object
sub_grade               object
emp_title               object
emp_length              object
home_ownership          object
annual_inc             float64
verification_status     object
issue_d                 object
loan_status             object
purpose                 object
addr_state              object
dti                    float64
fico_score             float64
dtype: object


In [9]:
# Display the number of unique values in each column
for a in clean_accept_df:
    print(a, len(clean_accept_df[a].unique()))

loan_amnt 1573
term 3
int_rate 674
installment 93302
grade 8
sub_grade 36
emp_title 512695
emp_length 12
home_ownership 7
annual_inc 89369
verification_status 4
issue_d 140
loan_status 10
purpose 15
addr_state 52
dti 10846
fico_score 49




**Data Wrangling and Cleaning for Loan Rejected Dataset**



In [10]:
# Reads in the rejected data : 

obj = s3.get_object(Bucket='davis-data-cloud-of-wonders', Key='rejected_2007_to_2018Q4.csv')
data = obj['Body'].read().decode('utf-8')

# Validates whether the connection to s3 is successfull or fail 
status = obj.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 put_object response. Status - {status}")
else:
    print(f"Unsuccessful S3 put_object response. Status - {status}")

# Use StringIO to convert the string data to a file-like object
data_file = StringIO(data)

# Create a DataFrame from the CSV data
rejected_df = pd.read_csv(data_file)

rejected_df.head()

Successful S3 put_object response. Status - 200


,amt_requested,date,purpose,risk_score,dti,zip_code,state,employment_length,policy_code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [11]:
# view number of columns and rows
rejected_df.shape

(27648741, 9)

In [12]:
# Drop zip code from column
clean_reject_df = rejected_df.drop(columns=["zip_code"])
clean_reject_df.head()

,amt_requested,date,purpose,risk_score,dti,state,employment_length,policy_code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,MD,< 1 year,0.0


In [19]:
# Renaming rejected Columns
clean_reject_df = clean_reject_df.rename(columns= {'amt_requested':'loan_amnt', 
                                                   'date': 'issue_d',
                                                   'risk_score':'fico_score',
                                                   'employment_length': 'emp_length'
                                                   })
clean_reject_df.head()

,loan_amnt,issue_d,purpose,fico_score,dti,state,emp_length,policy_code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,MD,< 1 year,0.0


In [20]:
# view the dataset shape after removal of un-needed columns:
clean_reject_df.shape

(27648741, 8)

In [21]:
# display the datatypes for clean_reject_df 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(clean_reject_df.dtypes)

loan_amnt      float64
issue_d         object
purpose         object
fico_score     float64
dti             object
state           object
emp_length      object
policy_code    float64
dtype: object


In [28]:
# I've noticed that clean_reject_df dti datatype was an object instead of float
# I've converted using rstrip to remove the "%" and calculate as numeric float value / 100
clean_reject_df['dti'] = clean_reject_df['dti'].str.rstrip("%").astype(float)/100


In [29]:
# Confirmed that the dti datatype is converted to an float64
clean_reject_df.dtypes

loan_amnt      float64
issue_d         object
purpose         object
fico_score     float64
dti            float64
state           object
emp_length      object
policy_code    float64
dtype: object

In [22]:
# Displaying each column unique values for the clean_reject_df datasets 
for b in clean_reject_df:
    print(b, len(clean_reject_df[b].unique()))

loan_amnt 3640
issue_d 4238
purpose 73928
fico_score 693
dti 126145
state 52
emp_length 12
policy_code 3




**Merging two datasets**


In [30]:
# quick review of the clean_accept_df dataset
clean_accept_df.head()


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,fico_score
0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,PA,5.91,677.0
1,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,SD,16.06,717.0
2,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,home_improvement,IL,10.78,697.0
3,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,debt_consolidation,NJ,17.06,787.0
4,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,major_purchase,PA,25.37,697.0


In [31]:
# quick review of the clean_reject_df dataset
clean_reject_df.head()

,loan_amnt,issue_d,purpose,fico_score,dti,state,emp_length,policy_code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,0.1000,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,0.1000,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,0.1000,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,0.3864,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,0.0943,MD,< 1 year,0.0


In [48]:
# Create new column loan_app_id to use as an identifier as index
clean_accept_df['loan_app_id'] = range(1, len(clean_accept_df) + 1)
clean_reject_df['loan_app_id'] = range(1, len(clean_reject_df) + 1)


In [50]:
clean_reject_df.set_index('loan_app_id')


,loan_amnt,issue_d,purpose,fico_score,dti,state,emp_length,policy_code
loan_app_id,,,,,,,,
1,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,0.1000,NM,4 years,0.0
2,1000.0,2007-05-26,Consolidating Debt,703.0,0.1000,MA,< 1 year,0.0
3,11000.0,2007-05-27,Want to consolidate my debt,715.0,0.1000,MD,1 year,0.0
4,6000.0,2007-05-27,waksman,698.0,0.3864,MA,< 1 year,0.0
5,1500.0,2007-05-27,mdrigo,509.0,0.0943,MD,< 1 year,0.0
...,...,...,...,...,...,...,...,...
27648737,10000.0,2016-12-31,Debt consolidation,590.0,0.4126,OH,< 1 year,0.0
27648738,10000.0,2016-12-31,moving,NaN,0.0148,MD,5 years,0.0
27648739,1200.0,2016-12-31,Other,686.0,0.1026,CA,< 1 year,0.0


In [51]:
clean_accept_df.set_index('loan_app_id')

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,fico_score
loan_app_id,,,,,,,,,,,,,,,,,
1,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,PA,5.91,677.0
2,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,SD,16.06,717.0
3,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,home_improvement,IL,10.78,697.0
4,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,debt_consolidation,NJ,17.06,787.0
5,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,major_purchase,PA,25.37,697.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260697,40000.0,60 months,10.49,859.56,B,B3,Vice President,9 years,MORTGAGE,227000.0,Verified,Oct-2016,Current,debt_consolidation,CA,12.75,707.0
2260698,24000.0,60 months,14.49,564.56,C,C4,Program Manager,6 years,RENT,110000.0,Not Verified,Oct-2016,Charged Off,debt_consolidation,FL,18.30,662.0
2260699,14000.0,60 months,14.49,329.33,C,C4,Customer Service Technician,10+ years,MORTGAGE,95000.0,Verified,Oct-2016,Current,debt_consolidation,TX,23.36,662.0


In [62]:
# Joined both datasets together, "concat" using join inner on "loan_request_id" column 
joined_loan_df = pd.merge(clean_accept_df, clean_reject_df, on='loan_app_id', how='inner')
joined_loan_df.sample(n=5)

,loan_amnt_x,term,int_rate,installment,grade,sub_grade,emp_title,emp_length_x,home_ownership,annual_inc,...,fico_score_x,loan_app_id,loan_amnt_y,issue_d_y,purpose_y,fico_score_y,dti_y,state,emp_length_y,policy_code
1606793,40000.0,60 months,16.01,972.94,C,C5,"President , CWALocal 1182",10+ years,RENT,155000.0,...,702.0,1606794,15000.0,2016-06-22,debt_consolidation,NaN,0.3576,NJ,< 1 year,0.0
760170,40000.0,36 months,6.67,1229.06,A,A2,Physician,9 years,OWN,300000.0,...,807.0,760171,3000.0,2016-04-01,debt_consolidation,NaN,0.1150,MI,NaN,0.0
145496,13650.0,60 months,13.99,317.55,C,C4,Accounting specialist,3 years,RENT,52000.0,...,662.0,145497,5000.0,2010-08-08,vacation,482.0,0.2093,IL,< 1 year,0.0
1278906,19200.0,36 months,14.99,665.49,C,C5,Physical Therapist,2 years,RENT,75000.0,...,667.0,1278907,20000.0,2016-05-12,other,NaN,0.5224,NC,< 1 year,0.0
312712,14000.0,36 months,10.99,458.28,B,B4,Owner,10+ years,RENT,50000.0,...,662.0,312713,13000.0,2011-07-30,debt_consolidation,702.0,0.2046,FL,< 1 year,0.0


In [65]:
# checked the combined dataframe shape for total of rows and columns
joined_loan_df.shape

(2260701, 26)

In [58]:
# confirmed and verified the datatypes are correct
joined_loan_df.dtypes

loan_amnt_x            float64
term                    object
int_rate               float64
installment            float64
grade                   object
sub_grade               object
emp_title               object
emp_length_x            object
home_ownership          object
annual_inc             float64
verification_status     object
issue_d_x               object
loan_status             object
purpose_x               object
addr_state              object
dti_x                  float64
fico_score_x           float64
loan_app_id              int64
loan_amnt_y            float64
issue_d_y               object
purpose_y               object
fico_score_y           float64
dti_y                  float64
state                   object
emp_length_y            object
policy_code            float64
dtype: object

In [67]:
# Confirmed the combined unique datasets
for b in joined_loan_df:
    print(b, len(joined_loan_df[b].unique()))

loan_amnt_x 1573
term 3
int_rate 674
installment 93302
grade 8
sub_grade 36
emp_title 512695
emp_length_x 12
home_ownership 7
annual_inc 89369
verification_status 4
issue_d_x 140
loan_status 10
purpose_x 15
addr_state 52
dti_x 10846
fico_score_x 49
loan_app_id 2260701
loan_amnt_y 2158
issue_d_y 2177
purpose_y 61365
fico_score_y 561
dti_y 39796
state 52
emp_length_y 12
policy_code 2


**Write Output files back to s3**

--Once the dataset is formatted wrangled and cleaned we're outputting the data

In [ ]:

# This is an dummy dataframe, i've created to test to see if able to write data into AWS s3 output_file folder 

""" 
data1 = {'ID': [1, 2, 3, 4], 'Name': ['tom','nick','juli','solyiah'], 'Age' : [10, 15, 14, 10]}
data2 = {'ID': [5, 6, 7, 8], 'Name': ['dick', 'joe', 'harry', 'jake'], 'Age': [21, 30, 45, 30]}

test_df_1 = pd.DataFrame(data1)
test_df_2 = pd.DataFrame(data2)

joined_test_df = pd.merge(test_df_1, test_df_2)

joined_test_df.head()

# test_df_2.head()
# test_df_1.head()

"""

In [64]:
# variables for buck name and path to save the output file to
bucket_name = "davis-data-cloud-of-wonders"
path = "joined_loan_output_data.csv"

# converts string into file format before saving the file
with io.StringIO() as csv_buffer:
    joined_loan_df.to_csv(csv_buffer, index=False)

    # place file into the specified buck and path location in the s3 directory
    response = s3.put_object(
        Bucket= bucket_name, Key=path, Body=csv_buffer.getvalue()
    )

    # Validates whether the connection to s3 is successfull or fail 
    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200
